## ETL Pipeline

In [145]:
import sys
import pandas as pd
from sqlalchemy import create_engine

def load_merge_data(messages_filepath, categories_filepath):

    messages_filepath = 'data/disaster_messages.csv'
    categories_filepath = 'data/disaster_categories.csv'

    #Reads disaster_messages.csv and drop the original column
    df_mess = pd.read_csv(messages_filepath, encoding='latin-1')
    df_mess.drop(['original'],axis=1,inplace=True)
    
    #Reads disaster_categories.csv
    df_cat = pd.read_csv(categories_filepath, encoding='latin-1')

    # Merges both dataframes on ['Id']
    df = df_mess.merge(df_cat, how='outer', on=['id'])
    
    return df

In [146]:
def clean_data(df):
    
### Creates columns with correspondent values of the 'categories' column

    # Provides a list with all the columns extracted from the category column
    cat = df.loc[0,'categories']
    cat_list = cat.split(';')
    col_names = []
    for val in cat_list:
        c = val.split('-')[0]
        col_names.append(c)

    # Creates all columns in df with correct value
    for col in col_names[0:-1]:
        try:
            df[col]          = df['categories'].apply(lambda st: st[st.find("-")+1:st.find(";")])
            df['categories'] = df['categories'].str.split(';',n=1).str[1:]
            df['categories'] = df['categories'].apply(lambda x: str(x[0]))
        # deals with the last column
        except:
            df[col_names[-1]]= df['categories'].apply(lambda st: st[st.find("-")+1:])
    
    # Drops de 'categories' column
    df.drop(['categories'], axis = 1, inplace = True)
    
    #Remove duplicates
    print('Number of columns: {}, and number of duplicates: {}'.format(df['message'].shape[0],df[df.duplicated() == True]['id'].count()))
    print('The following rows are duplications:')
    print(df[['id','message']][df.duplicated()==True])
    df.drop_duplicates(inplace=True)
    #df.drop_duplicates(subset=['id', 'message'],inplace=True)
    print('Number of columns of new dataframe excluding duplicates: {}'.format(df['message'].shape[0]))
    
    return df

In [147]:
def main():

    # Load and merge datasets
    messages_filepath = 'data/disaster_messages.csv'
    categories_filepath = 'data/disaster_categories.csv'

    df = load_merge_data(messages_filepath, categories_filepath)

    # Clean and remove duplicates
    df = clean_data(df)

    # Save clean dataset into an sqlite database
    engine = create_engine('sqlite:///Disaster_response_pipelines.db')
    df.to_sql('Disaster_response_pipelines', engine, index=False)
    
main()

Number of columns: 26386, and number of duplicates: 172
The following rows are duplications:
          id                                            message
164      202  ?? port au prince ?? and food. they need gover...
165      202  ?? port au prince ?? and food. they need gover...
658      804  elle est vraiment malade et a besoin d'aide. u...
659      804  elle est vraiment malade et a besoin d'aide. u...
660      804  elle est vraiment malade et a besoin d'aide. u...
...      ...                                                ...
25291  29022  In a field in Jallouzai, just inside Pakistan,...
25292  29022  In a field in Jallouzai, just inside Pakistan,...
25378  29119  Most victims (90 per cent) show little or no s...
25379  29119  Most victims (90 per cent) show little or no s...
25380  29119  Most victims (90 per cent) show little or no s...

[172 rows x 2 columns]
Number of columns of new dataframe excluding duplicates: 26214
